In [ ]:
# Initialize QuantBook and add SPY equity
qb = QuantBook()
spy = qb.AddEquity("SPY")

# Retrieve 30 days of intraday hourly price history for SPY
history = qb.History(spy.Symbol, 100000, Resolution.Hour, extendedMarketHours=False)
history = history.loc[spy.Symbol].between_time('09:30', '16:00')
print(history.head(10))
# Group the data by the hour part of the timestamp and count the occurrences
hourly_counts = history.index.hour.value_counts().sort_index()
# Print the count for each hour from 10:00 to 16:00
for hour in range(10, 17):
    count = hourly_counts.get(hour, 0)  # Use .get() to handle missing hours
    print(f"Number of data points for {hour:02d}:00: {count}")

In [ ]:
import matplotlib.pyplot as plt
import scipy.stats as stats

# Define the pairs of hours to compare
time_pairs = [
    (10, 11), (10, 12), (10, 13), (10, 14), (10, 15), (10, 16),
    (11, 12), (11, 13), (11, 14), (11, 15), (11, 16),
    (12, 13), (12, 14), (12, 15), (12, 16),
    (13, 14), (13, 15), (13, 16),
    (14, 15), (14, 16),
    (15, 16)
]

# Dictionary to store percentage differences for each pair of intervals
pct_diffs = {}

# Calculate and store the percentage differences for each pair of hours
for hour1, hour2 in time_pairs:
    # Filter for the two target hours (hour1 and hour2)
    history_pair = history[(history.index.hour == hour1) | (history.index.hour == hour2)]
    
    # Calculate the percentage difference from hour1 to hour2 for each day
    pct_diff_pair = history_pair.groupby(history_pair.index.date).apply(
        lambda x: ((x.loc[x.index.hour == hour2, 'close'].values[0] - x.loc[x.index.hour == hour1, 'close'].values[0])
                   / x.loc[x.index.hour == hour1, 'close'].values[0]) * 100
        if hour2 in x.index.hour and hour1 in x.index.hour else None
    ).dropna()  # Drop days where data is missing
    
    # Store the results in the dictionary
    pct_diffs[f"{hour1}:00 to {hour2}:00"] = pct_diff_pair

# Plot histograms with a 4x6 grid
plt.figure(figsize=(20, 15))
for i, (interval, data) in enumerate(pct_diffs.items(), 1):
    plt.subplot(4, 6, i)
    plt.hist(data, bins=300, color='skyblue', edgecolor='black', alpha=0.7)
    plt.title(f"{interval} Percentage Difference")
    plt.xlabel("Percentage Price Difference (%)")
    plt.ylabel("Frequency")
    plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.suptitle("Distribution of SPY Percentage Price Differences Across Selected Time Intervals", y=1.02)
plt.show()



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

# List of distributions to test
distributions = [stats.norm, stats.t, stats.laplace, stats.cauchy, stats.expon]

# Function to fit a set of distributions and plot the best fit, displaying parameters in the legend
def fit_and_plot_distribution(data, title=""):
    best_fit_name = None
    best_fit_params = None
    best_ks_stat = np.inf
    best_dist = None

    plt.figure(figsize=(10, 6))
    
    # Plot the histogram of the data
    plt.hist(data, bins=300, density=True, color='skyblue', edgecolor='black', alpha=0.7, label='Data')
    
    # Test each distribution
    for distribution in distributions:
        # Fit the distribution to the data
        params = distribution.fit(data)
        
        # Perform the Kolmogorov-Smirnov test
        ks_stat, _ = stats.kstest(data, distribution.name, args=params)
        
        # Update the best fit if this distribution is better
        if ks_stat < best_ks_stat:
            best_fit_name = distribution.name
            best_fit_params = params
            best_ks_stat = ks_stat
            best_dist = distribution
            
        # Plot the PDF of the fitted distribution
        x = np.linspace(min(data), max(data), 100)
        plt.plot(x, distribution.pdf(x, *params), label=f"{distribution.name} (KS={ks_stat:.3f})")
    
    # Plot the best fit in a bold line and include parameters in the legend
    if best_dist:
        # Prepare parameters for legend display based on distribution type
        param_text = ""
        if best_fit_name == "norm":
            param_text = f"Mean={best_fit_params[0]:.4f}, Std Dev={best_fit_params[1]:.4f}"
        elif best_fit_name == "t":
            param_text = f"DF={best_fit_params[0]:.4f}, Mean={best_fit_params[1]:.4f}, Scale={best_fit_params[2]:.4f}"
        elif best_fit_name == "laplace":
            param_text = f"Mean={best_fit_params[0]:.4f}, Scale={best_fit_params[1]:.4f}"
        elif best_fit_name == "cauchy":
            param_text = f"Mean={best_fit_params[0]:.4f}, Scale={best_fit_params[1]:.4f}"
        elif best_fit_name == "expon":
            param_text = f"Location={best_fit_params[0]:.4f}, Scale={best_fit_params[1]:.4f}"
        
        # Plot the best-fit PDF with parameters in the legend
        plt.plot(x, best_dist.pdf(x, *best_fit_params), 'k-', linewidth=2, 
                 label=f"Best Fit: {best_fit_name} (KS={best_ks_stat:.3f})\n{param_text}")
    
    # Show legend and title
    plt.title(f"{title}\nBest fit: {best_fit_name} with KS statistic: {best_ks_stat:.3f}")
    plt.xlabel("Percentage Price Difference (%)")
    plt.ylabel("Density")
    plt.legend()
    plt.grid(True)
    plt.show()
    
    return best_fit_name, best_fit_params

# Example of using this function for each interval
best_fits = {}
for interval, data in pct_diffs.items():
    best_fit_name, best_fit_params = fit_and_plot_distribution(data, title=f"Distribution Fit for {interval}")
    best_fits[interval] = (best_fit_name, best_fit_params)

In [ ]:
def t_distribution_bounds(degrees_of_freedom, mean, scale, percent):
    # Calculate the tail probabilities for the specified central percentage
    alpha = (1 - percent / 100) / 2  # Split the remaining probability between two tails

    # Calculate the critical values for the lower and upper bounds in the standard t-distribution
    lower_bound = stats.t.ppf(alpha, degrees_of_freedom)
    upper_bound = stats.t.ppf(1 - alpha, degrees_of_freedom)

    # Adjust bounds for mean and scale
    lower_bound = mean + lower_bound * scale
    upper_bound = mean + upper_bound * scale

    return lower_bound, upper_bound

In [ ]:
# Sanity Check
degrees_of_freedom = 2.7274
mean = 0.0394
scale = 0.5488
percent = 95  # Target interval percentage

# Get the bounds
lower_bound, upper_bound = t_distribution_bounds(degrees_of_freedom, mean, scale, percent)
print(f"Calculated Bounds for {percent}%: ({lower_bound:.4f}, {upper_bound:.4f})")

# Now, find the actual percentage of data within these bounds
data = pct_diffs['10:00 to 16:00']  # Use the actual data

# Calculate the percentage of data within the bounds
within_bounds = data[(data >= lower_bound) & (data <= upper_bound)]
actual_percent_within_bounds = len(within_bounds) / len(data) * 100
print(f"Actual percentage of data within bounds: {actual_percent_within_bounds:.2f}%")

# Output the results
print(f"For a 90% interval, the calculated bounds are ({lower_bound:.4f}, {upper_bound:.4f}).")
print(f"The actual percentage of data within these bounds is {actual_percent_within_bounds:.2f}%.")